In [3]:
import numpy as np
import pandas as pd


# loding the messy data

In [4]:
#loding the messy data 

df = pd.read_csv("marketing_campaign_data_messy.csv")
print(df.head())


   Campaign_ID              Campaign_Name           Start_Date    End_Date  \
0     CMP-00001       Q4_Summer_CMP-00001  2023-11-24 00:00:00  2023-12-13   
1     CMP-00002       Q1_Launch_CMP-00002  2023-05-06 00:00:00  2023-05-12   
2     CMP-00003       Q3_Winter_CMP-00003  2023-12-13 00:00:00  2023-12-20   
3     CMP-00004  Q1_BlackFriday_CMP-00004           2023-10-30  2023-11-03   
4     CMP-00005       Q2_Winter_CMP-00005  2023-04-22 00:00:00  2023-04-23   

    Channel  Impressions  Clicks     Spend  Conversions Active  Clicks  \
0    TikTok        16795      197  $102.82         20.0      Y     NaN   
1  Facebook         1860       30    24.33          1.0      0     NaN   
2     Email        77820      843  1323.39         51.0     No     NaN   
3    TikTok        55886     2019  2180.38        135.0   True     NaN   
4  Facebook         7265      169   252.44         30.0    Yes     NaN   

  Campaign_Tag  
0           TI  
1           FA  
2           EM  
3           TI  
4

In [5]:
print(f"LOADED DATASET : {df.shape[0]} rows,{df.shape[1]} columns ")

LOADED DATASET : 2020 rows,12 columns 


# STEP 1
## CLeaning Header 


In [6]:
print(df.columns.tolist())

df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

print("FIX APPLIED")
print(df.columns.tolist())

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']
FIX APPLIED
['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


### 1)in the above we see the some colums have unnessecery space infont and backword of the column name so using  .str.strip() we remove the unnessecery space.

### 2) and also we see the above some colums name are starting letter are capital letter and after mid "_" this also Capital latter so using .str.lower() we make all the column name in the small letter.

### 3) and also if their is unnnesary sapce in the middel of a column name so we replace with the "-".

# STEP 2
## TYPE CONVERSION & CURRENCY CLEANING 

In [7]:
dirty_spend_mask = df['spend'].astype(str).str.contains(r'\$') #it shows the spend value with $ sign
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(3))

df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]','', regex =True)  # 1  it replace other than d=digit, . , - thes charecter to nothing
df['spend'] = pd.to_numeric(df['spend']) # 2  to Numeric data tupe of column

print("FIX APPLIED ")
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(3))

   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22
FIX APPLIED 
   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22


## 1  it replace other than d=digit, . , - thes charecter to nothing
## 2  to Numeric data tupe of column

### in the above we remove the doller sign from the "spend" column value wich have the doller sign with them 
### firstly seeing the value with doller sign , we first convert the spend column into string and check the doller sing and replce / remove them and reconvert the spend column to numeric 

# STEP 3
## CATEGORICAL TYPOS

In [8]:
print(df['channel'].unique())

cleanup_map = {
    'Facebok':'Facebook',
    'Insta_gram':'Instagram',
    'Gogle':'Google Ads',
    'Tik_Tok':'TikTok',
    'E-mail':'Email' ,
    'N/A': np.nan
}

df['channel'] = df['channel'].replace(cleanup_map)

print ("FIX APPLIED")
print(df['channel'].unique())

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan 'Gogle'
 'Tik_Tok' 'Facebok' 'Insta_gram']
FIX APPLIED
['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' nan]


### in the above "channel " column theri is TYPOS like TikTok - Tik_Tok,Instagram -Insta_gram ,Email - E-mail etc
### in cleanup_mape we store the incorrect str name with righ str name and replace all incorrect typo string with correct str 
### and retrive the new replae correct str unique value

# STEP 4
## HANDLING THE MIX BOOLEANS

In [10]:
print(df['active'].unique())

bool_map = {
          'Yes' : 'True',
           'Y' : 'True',
           '1'  : 'True',
           'No' : 'False',
            '0': 'False',
            0 : 'False'
}

df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)

print("FIX APPLIED")
print(df['active'].unique())

['Y' '0' 'No' 'True' 'Yes' '1' 'False']
FIX APPLIED
[ True False]


### in the active ther are multple values theat saye only two thing but that have multipl value to say that
### we remove unnecessary thing and keep only meanigful and correct one .


# STEP 5
## DATE PARSING

In [12]:
print(df['start_date'].dtype) # it has a object data type 

df['start_date'] = pd.to_datetime(df['start_date'],errors = 'coerce')
df['end_date'] = pd.to_datetime(df['end_date'],errors = 'coerce')

print("FIX APPLIED")
print(df['start_date'].dtype)

object
FIX APPLIED
datetime64[ns]


### id "start_date & end_date " has a (OBJECT) data type so we shuld convert that into a (datetime) data type using above code we convert that 

# STEP 6 
# REMOVE DUPLICATE COLUMNS

In [21]:
df = df.loc[:,~df.columns.duplicated()]

In [26]:
df

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24,2023-12-13,TikTok,16795,197,102.82,20.0,True,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06,2023-05-12,Facebook,1860,30,24.33,1.0,True,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13,2023-12-20,Email,77820,843,1323.39,51.0,True,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,NaT,2023-11-03,TikTok,55886,2019,2180.38,135.0,False,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22,2023-04-23,Facebook,7265,169,252.44,30.0,True,FA
...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31,2023-11-13,TikTok,30592,586,503.95,77.0,True,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01,2023-09-26,Google Ads,20097,897,1641.00,162.0,True,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09,2023-02-21,Instagram,33254,1117,883.82,214.0,True,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30,2023-04-27,Facebook,68728,2960,4198.50,591.0,True,FA


# STEP 7
## LOGICAL INTEGRITY ( CLICKS VS IMPRESSION)

In [30]:
impossible_mask = df['clicks'] > df['impressions']
print(df.loc[impossible_mask,['campaign_id','impressions','clicks']].head(3))

Empty DataFrame
Columns: [campaign_id, impressions, clicks]
Index: []


# STEP 8
## LOGICAL INTEGRITY (TIME TRAVEL)

In [38]:
time_travel_mask = df['end_date'] < df['start_date']
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3))

df.loc[time_travel_mask ,'end_date'] = df.loc[time_travel_mask ,'start_date'] + pd.Timedelta(days = 30)

print("FIX APPLIED")
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3))

   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-05-01
54   CMP-00055 2023-09-01 2023-08-27
71   CMP-00072 2023-02-01 2023-01-27
FIX APPLIED
   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-06-05
54   CMP-00055 2023-09-01 2023-10-01
71   CMP-00072 2023-02-01 2023-03-03


### in the above time travel intefgrity means the end_date is befor the start_date how it possible (like time travel)
### so we change that and ,make sure the end_date is minimun 30day after the start_day

# STEP 9 
# HANSLING OUTLIERS

In [45]:
Q1 = df['spend'].quantile(0.25)
Q3 = df['spend'].quantile(0.75)

IQR = Q3 - Q1
upper_limit = Q3 + (3*IQR)

outlier_mask = df['spend'] >upper_limit
print(df.loc[outlier_mask,['campaign_id','spend']].head(3))

print("FIX APPLIED")
df.loc[outlier_mask,'spend'] = upper_limit;
print(df.loc[outlier_mask,['campaign_id','spend']].head(3))

     campaign_id      spend
789    CMP-00790  500000.00
1443   CMP-01444    8921.51
1460   CMP-01461  500000.00
FIX APPLIED
     campaign_id      spend
789    CMP-00790  8603.5375
1443   CMP-01444  8603.5375
1460   CMP-01461  8603.5375


# STEP 10 
## STRING PARSING

In [55]:
print(df['campaign_name'].head(3))

df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')

print("FIX APLLIED")
print(df[['campaign_name','season']].head(3))


0    Q4_Summer_CMP-00001
1    Q1_Launch_CMP-00002
2    Q3_Winter_CMP-00003
Name: campaign_name, dtype: object
FIX APLLIED
         campaign_name  season
0  Q4_Summer_CMP-00001  Summer
1  Q1_Launch_CMP-00002  Launch
2  Q3_Winter_CMP-00003  Winter
